In [1]:
import findspark

findspark.init("/opt/cloudera/parcels/CDH-6.2.1-1.cdh6.2.1.p0.1425774/lib/spark")
import pyspark

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("product-analysis").setMaster("local")
sc = SparkContext(conf=conf)


In [45]:
input ="sales.csv"
sales= sc.textFile(input)

In [46]:
sales.collect()

['date,product_id,customer_id,amount',
 '2021-04-05,P1,C1,42.94',
 '2021-09-17,P1,C1,87.85',
 '2021-01-11,P2,C5,22.77',
 '2021-04-03,P2,C3,83.44',
 '2021-08-04,P3,C2,77.37',
 '2021-07-23,P5,C2,40.62',
 '2021-12-26,P2,C5,80.86',
 '2021-03-30,P3,C1,84.64',
 '2021-09-26,P1,C3,48.19',
 '2021-03-12,P3,C1,21.77',
 '2021-07-19,P1,C4,42.28',
 '2021-05-17,P4,C3,49.7',
 '2021-04-20,P2,C2,17.78',
 '2021-08-01,P3,C1,20.35',
 '2021-02-13,P2,C4,62.56',
 '2021-05-20,P2,C5,47.06',
 '2021-07-27,P4,C5,58.42',
 '2021-02-25,P4,C1,67.02',
 '2021-07-01,P1,C2,44.32',
 '2021-11-01,P5,C1,89.41',
 '2021-09-30,P1,C1,70.65',
 '2021-05-03,P5,C2,84.29',
 '2021-08-31,P5,C4,63.68',
 '2021-05-11,P1,C2,44.2',
 '2021-10-07,P5,C3,58.37',
 '2021-06-18,P3,C2,50.37',
 '2021-05-04,P3,C4,82.5',
 '2021-03-12,P5,C4,12.59',
 '2021-09-01,P4,C5,98.88',
 '2021-02-18,P3,C4,82.23',
 '2021-03-22,P4,C4,61.24',
 '2021-03-27,P1,C4,67.22',
 '2021-04-09,P2,C3,55.98',
 '2021-01-26,P5,C3,53.49',
 '2021-05-16,P4,C4,28.86',
 '2021-11-13,P4,C5,

In [47]:
header = sales.first()
sales = sales.filter(lambda x : x!=header)
sales.take(5)

['2021-04-05,P1,C1,42.94',
 '2021-09-17,P1,C1,87.85',
 '2021-01-11,P2,C5,22.77',
 '2021-04-03,P2,C3,83.44',
 '2021-08-04,P3,C2,77.37']

In [48]:
sales = sales.map(lambda x: x.split(","))
sales.take(5)

[['2021-04-05', 'P1', 'C1', '42.94'],
 ['2021-09-17', 'P1', 'C1', '87.85'],
 ['2021-01-11', 'P2', 'C5', '22.77'],
 ['2021-04-03', 'P2', 'C3', '83.44'],
 ['2021-08-04', 'P3', 'C2', '77.37']]

In [49]:
sales_pol= sales.map(lambda x: (x[1], (float(x[3]))))
sales_pol.collect()

[('P1', 42.94),
 ('P1', 87.85),
 ('P2', 22.77),
 ('P2', 83.44),
 ('P3', 77.37),
 ('P5', 40.62),
 ('P2', 80.86),
 ('P3', 84.64),
 ('P1', 48.19),
 ('P3', 21.77),
 ('P1', 42.28),
 ('P4', 49.7),
 ('P2', 17.78),
 ('P3', 20.35),
 ('P2', 62.56),
 ('P2', 47.06),
 ('P4', 58.42),
 ('P4', 67.02),
 ('P1', 44.32),
 ('P5', 89.41),
 ('P1', 70.65),
 ('P5', 84.29),
 ('P5', 63.68),
 ('P1', 44.2),
 ('P5', 58.37),
 ('P3', 50.37),
 ('P3', 82.5),
 ('P5', 12.59),
 ('P4', 98.88),
 ('P3', 82.23),
 ('P4', 61.24),
 ('P1', 67.22),
 ('P2', 55.98),
 ('P5', 53.49),
 ('P4', 28.86),
 ('P4', 75.6),
 ('P4', 40.3),
 ('P4', 79.73),
 ('P4', 24.57),
 ('P5', 78.8),
 ('P4', 85.0),
 ('P1', 41.37),
 ('P2', 41.87),
 ('P4', 68.85),
 ('P4', 95.06),
 ('P3', 37.41),
 ('P5', 75.43),
 ('P2', 67.75),
 ('P4', 82.18),
 ('P1', 51.61),
 ('P5', 96.55),
 ('P5', 87.48),
 ('P1', 16.19),
 ('P4', 16.62),
 ('P2', 30.04),
 ('P3', 56.12),
 ('P2', 65.12),
 ('P4', 51.24),
 ('P2', 82.09),
 ('P3', 20.69),
 ('P1', 39.17),
 ('P4', 30.24),
 ('P5', 37.67),

In [50]:
sales_po_pl= sales_pol.reduceByKey(lambda x, y: x+y)
sales_po_pl.collect()

[('P1', 112660.08000000015),
 ('P2', 111657.62000000013),
 ('P3', 109063.67000000016),
 ('P5', 108856.43999999996),
 ('P4', 110930.96000000008)]

In [55]:
output_file = "product_sales"
sales_po_pl.map(lambda x: ",".join([str(e) for e in x])).saveAsTextFile(output_file)